In [ ]:
import io, os, sys, types
from IPython import get_ipython
from Utils import NotebookFinder
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sys.meta_path.append(NotebookFinder())

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        

In [ ]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    

In [ ]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())


In [ ]:
def null(*args):
    pass

def setDbgPrint(dbgPrint):
    if type(dbgPrint) == str:
        dbgPrint = eval(dbgPrint)
        
    if dbgPrint is None or dbgPrint in ['', {}]:
        dbgPrint = lambda *args: None
    
    return dbgPrint

def printPeriodic(*args, **kwargs):
    """Print periodically per optional arguments 'period' & 'counter' when 'counter % period == 0',
       else print normally.
    """
    period = None
    if 'period' in kwargs:
        period = kwargs.pop('period')
    
    counter = None
    if 'counter' in kwargs:
        counter = kwargs.pop('counter')
    
    isPrint = True
    if counter and period:
        if counter % period != 0:
            isPrint = False
    
    if isPrint:
        print(*args, **kwargs)



In [ ]:
from env import Env
from RL_brain import DeepQNetwork
from  Configuration import null
from Configuration import setDbgPrint
from runs import sim
from testRun import testRun
import tensorflow as tf

In [ ]:
def run(cfg=None):
    # For ease of access to run parameters.
    step = 0
    numEpisodes = cfg['numEpisodesTrain']
    maxSteps = cfg['maxStepsTrain']
    startLearnStep = cfg['startLearnStep']
    learnPeriod = cfg.get('learnPeriod', 5)
    dbgPrint = cfg.get('dbgPrint', null)
    statusPeriod = cfg.get('statusPeriod', 1)

    TPcount = 0
    FPcount = 0
    TNcount = 0
    FNcount = 0
    Pr_history = []
    Rc_history = []
    F1_history =[]
    for episode in range(1,numEpisodes+1):
        # initial observation
        step = 0
        env_state_1 = env.reset()
        env.count_history = []
        while step < maxSteps:
            # fresh env
            # Output current observation:
            #print('{}: current env = {}\n'.format(env.time, env.__dict__))

            env_state = env_state_1
            # RL choose action based on env_state
            env_state = np.hstack(env_state)
            action = RL.choose_action(env_state)
         #   dbgPrint('{}: action_ = {}, observation = {}\n'.format(env.time, action_, observation), period=statusPeriod, counter=env.time)
            # print(observation)
            # RL take action and get next observation and reward
            env_state_, observation_, reward,label = env.step(action)
            if label != 0 and action != 0:
                TPcount += 1
            elif action != 0 and label == 0:
                FPcount += 1
            elif action == 0 and label == 0:
                TNcount += 1
            elif action == 0 and label != 0:
                FNcount += 1
            if ((TPcount + FNcount) != 0 and (TPcount + FPcount) != 0):
                Pr = TPcount / (TPcount + FPcount)
                Rc = TPcount / (TPcount + FNcount)
                F1 = 2/(1/Pr+1/Rc)
                Pr_history.append(TPcount / (TPcount + FPcount))#It is the ratio of correctly classified attacks flows (TP), in front of all the classified flows (TP+FP).
                Rc_history.append(TPcount / (TPcount + FNcount))#It is the ratio of correctly classified attack flows (TP), in front of all generated flows (TP+FN).
                F1_history.append(2/(1/Pr+1/Rc))  #It is a harmonic combination of the precision and recall into a single measure.
                dbgPrint('Pr = {},Rc = {},F1 = {}'.format(Pr,Rc,F1),period = statusPeriod,counter=env.time)
            RL.store_transition(env_state, action, reward, observation_)
            # dbgPrint("env_state = {},label={}, action = {},reward = {}\n".format(env_state,label,action,reward), period=statusPeriod,
            #          counter=env.time)
           # print("env_state = {},label={}, action = {},reward = {},observation_ ={}\n".format(env_state,label,action,reward,observation_))
            if (step > startLearnStep) and (step % learnPeriod == 0):
                RL.learn()

            # swap observation
            #observation = observation_
            env_state_1 = observation_

            # break while loop when end of this episode
            step += 1

    

    plt.figure()
    plt.subplot(131)
    plt.plot(
       Pr_history, '.-')
    plt.xlabel('Training Step')
    plt.ylabel('Pr')

    plt.subplot(132)
    plt.plot(
        Rc_history, '.-')
    plt.xlabel('Training Step')
    plt.ylabel('Rc')

    plt.subplot(133)
    plt.plot(
        F1_history, '.-')
    plt.xlabel('Training Step')
    plt.ylabel('F1')
    plt.grid(True)
    plt.show()

    

In [ ]:
if __name__ == "__main__":
    #Generating data
   # generator()
    # For ease of access to env config. parameters.
    statusPeriod = sim['cfg']['run']['statusPeriod']
    dbgPrint = setDbgPrint(sim['cfg']['run'].get('dbgPrint'))
    test = sim['cfg']['run']['test']

    
    if test == 1:
        testRun()
    else:
    
        # Init. environment.
        env = Env(log=dbgPrint, statusPeriod=statusPeriod)

        # For ease of access to DQN config. parameters.
        learningRate = sim['cfg']['dqn']['learningRate']
        rewardDecay = sim['cfg']['dqn']['rewardDecay']
        eGreedy = sim['cfg']['dqn']['eGreedy']
        e_greedy_increment = sim['cfg']['dqn']['eGreedyincrement']
        replaceTargetIter = sim['cfg']['dqn']['replaceTargetIter']
        rewardDecay = sim['cfg']['dqn']['learningRate']
        memorySize = sim['cfg']['dqn']['memorySize']

        RL = DeepQNetwork(n_actions=env.n_actions,
                          n_features=env.n_features,
                          learning_rate=learningRate,
                          reward_decay=rewardDecay,
                          e_greedy=eGreedy,
                          replace_target_iter=replaceTargetIter,
                          e_greedy_increment = e_greedy_increment,
                          memory_size=memorySize,
                          dbgPrint=dbgPrint,
                          statusPeriod=statusPeriod
                          )
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        run(cfg=sim['cfg']['run'])

In [ ]:
if 'RL' in locals().keys():
   # allow use of interactive plot controls, e.g., zoom
    get_ipython().run_line_magic('matplotlib', 'notebook')
    #RL_plot_cost = RL.plot_cost()
    #RL_plot_cost.gca().grid(True)


  

    # allow use of interactive plot controls, e.g., zoom
    get_ipython().run_line_magic('matplotlib', 'notebook')
    # env.plot_count()
    plt.figure()
    plt.plot(
        np.divide( np.cumsum([1 if i >= 10 else 0 for i in env.count_history]), np.arange(len(env.count_history)) + 1),
        '.-')
    plt.xlabel('Training Step')
    plt.ylabel('Correct rate')
    plt.grid(True)
    plt.show()